In [1]:
import json
import os

with open("retrieval/wiki.json", "r") as f:
    wiki_sents = json.loads(f.read())



In [2]:
wiki_sents = {int(k) : v for k, v in wiki_sents.items()}
wiki_chunks = {k : " ".join(v) for k, v in wiki_sents.items()}  # https://arxiv.org/abs/2109.08133
wiki_chunks

{0: "The Universe is all of time and space and its contents. The Universe includes planets, stars, galaxies, the contents of intergalactic space, the smallest subatomic particles, and all matter and energy. The observable universe is about 28 billion parsecs (91 billion light-years) in diameter at the present time. The size of the whole Universe is not known and may be either finite or infinite. Observations and the development of physical theories have led to inferences about the composition and evolution of the Universe. Throughout recorded history, cosmologies and cosmogonies, including scientific models, have been proposed to explain observations of the Universe. The earliest quantitative geocentric models were developed by ancient Greek philosophers and Indian philosophers. Over the centuries, more precise astronomical observations led to Nicolaus Copernicus's heliocentric model of the Solar System and Johannes Kepler's improvement on that model with elliptical orbits, which was e

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'BAAI/bge-small-en'
device = "cuda"

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
import numpy as np
from tqdm import tqdm

out_path = f"retrieval/wiki_embeddings_{model_name.split('/')[-1]}.npy"
save_interval = 10000
wiki_chunks_list = list(wiki_chunks.values())
batch_size = 1000
sentence_embeddings = np.zeros((len(wiki_chunks_list), model.config.hidden_size))
for i in tqdm(range(0, len(wiki_chunks_list), batch_size)):
    # Tokenize sentences
    encoded_chunks = tokenizer(wiki_chunks_list[i:i + batch_size], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_chunks.to(device))
        # Perform pooling. In this case, cls pooling.
        embeddings = model_output[0][:, 0]
    # normalize embeddings
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    sentence_embeddings[i:i + batch_size, :] = embeddings.cpu().numpy()

    if i % save_interval == 0:
        np.save(out_path, sentence_embeddings)
        print(f"Saved embeddings for {i} sentences out of {len(wiki_chunks_list)}")
np.save(out_path, sentence_embeddings)
print(f"Saved embeddings for all sentences")

  0%|          | 0/3349 [00:00<?, ?it/s]

  0%|          | 1/3349 [00:07<6:58:29,  7.50s/it]

Saved embeddings for 0 sentences out of 3348807


  0%|          | 11/3349 [00:50<4:32:30,  4.90s/it]

Saved embeddings for 10000 sentences out of 3348807


...

100%|█████████▉| 3341/3349 [6:35:53<01:14,  9.37s/it]

Saved embeddings for 3340000 sentences out of 3348807


100%|██████████| 3349/3349 [6:36:54<00:00,  7.11s/it]


Saved embeddings for all sentences


: 

In [10]:
sim = torch.mm(query_embeddings, sentence_embeddings.transpose(0, 1))
topk = torch.topk(sim, 5, dim=1).indices.tolist()
topk

[[919], [517]]

In [11]:
# # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
instruction = "What information is relevant to determining whether the following text is true?"
queries = ["The Tang dynasty (618-907 CE) saw the development of a vibrant national identity through song and dance performance", "Modern Chinese musicians continue to explore new avenues in traditional music, and have collaborated with Western artists to create exciting works that bridge Eastern and Western forms of expression"]
encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')


'Buddhist art is the artistic practices that are influenced by Buddhism. It includes art media which depict Buddhas, bodhisattvas, and other entities; notable Buddhist figures, both historical and mythical; narrative scenes from the lives of all of these; mandalas and other graphic aids to practice; as well as physical objects associated with Buddhist practice, such as vajras, bells, stupas and Buddhist temple architecture. Buddhist art originated on the Indian subcontinent following the historical life of Siddhartha Gautama, 6th to 5th century BC, and thereafter evolved by contact with other cultures as it spread throughout Asia and the world. Buddhist art followed believers as the dharma spread, adapted, and evolved in each new host country. It developed to the north through Central Asia and into Eastern Asia to form the Northern branch of Buddhist art, and to the east as far as Southeast Asia to form the Southern branch of Buddhist art. In India, Buddhist art flourished and co-devel

In [13]:
embed_size

384

In [ ]:
import faiss

model_name = "bge-small-en"
dim = {"bge-small-en": 1024, "bge-small-en": 384}[model_name]

# use inner product metric
config = faiss.GpuIndexFlatConfig()
config.useFloat16 = True
config.device = 1
res = faiss.StandardGpuResources()
quantizer = faiss.GpuIndexFlatIP(res, dim, config)
nlist = 100  # number of clusters
m = 8  # number of subquantizers
index = faiss.IndexIVFPQ(quantizer, dim, nlist, m, 8)

# index = faiss.IndexFlatIP(dims)

# cpu_index = faiss.IndexFlatIP(1024)

# index = faiss.index_cpu_to_all_gpus(  # build the index
#     cpu_index
# )

# TODO: take a random sample of 10 files to train with
n_samples = 5
files = np.random.choice(os.listdir(embeddings_dir), n_samples)
vecs = []
for file in files:
    vecs.append(np.load(os.path.join(embeddings_dir, file)).astype(np.float16))
vecs = np.concatenate(vecs)
index.train(vecs)
print(index.is_trained)

False


In [ ]:
import os
import numpy as np
from tqdm import tqdm

embeddings_dir = f"retrieval/20230601.en.wiki_embeddings_{model_name}"
files = os.listdir(embeddings_dir)
# make sure to add keys to index in order
files = sorted(files, key=lambda file: int(file.split(".")[0]))
print(len(files))
for file in tqdm(files, desc="Adding embeddings to index"):
    batch_num = int(file.split(".")[0])
    vecs = np.load(os.path.join(embeddings_dir, file)).astype(np.float16)
    index.add(vecs)

176


Adding embeddings to index: 100%|██████████| 176/176 [15:50<00:00,  5.40s/it]


In [ ]:
vecs = np.load(os.path.join(embeddings_dir, files[-2]))

In [ ]:
query = vecs[:1, :]

In [ ]:
num_passages = 1
D, I = index.search(query, num_passages)
D, I

(array([[0.16384947]], dtype=float32), array([[34800000]]))

In [ ]:
from Retriever import Retriever
retriever = Retriever(
    use_IVF=True,
    encoder_name="BAAI/bge-small-en",
    pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
)

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model... done.


In [ ]:
retriever("""First, decide whether the statement can even be given an accuracy label, writing "Determinable" if it can, and "N/A" if the statement doesn't make any claims that can be judged on their accuracy. Then, if the statement is "Determinable",  rate the extent to which it is true, accurate, and not misleading. Your rating must be one of "N/A", "Determinable - Certainly True", "Determinable - True", "Determinable - Uncertain", "Determinable - False", "Determinable - Certainly False". Ratings should be based on the quality of the new information introduced in a statement, and if anything in the statement is incorrect or misleading, the statement is considered incorrect. "Other popular hashing algorithms include MD5, SHA-1, and RSA[[1]]\"""", threshold=0.)

(['misses important safeguards. It does not check A, B, and U are not zero.\n\nbased on http://srp.stanford.edu/design.html\n"""\nimport hashlib\nimport random\n\n# Note: str converts as is, str([1,2,3,4]) will convert to "[1,2,3,4]"\ndef H(*args) -> int:\n    """A one-way hash function."""\n    a = ":".join(str(a) for a in args)\n    return int(hashlib.sha256(a.encode("utf-8")).hexdigest(), 16)\n\ndef cryptrand(n: int = 1024):\n    return random.SystemRandom().getrandbits(n) % N\n\n# A large safe prime (N = 2q+1, where q is prime)\n# All arithmetic is done modulo N\n# (generated using "openssl dhparam -text 1024")\nN = """00:c0:37:c3:75:88:b4:32:98:87:e6:1c:2d:a3:32:\n       4b:1b:a4:b8:1a:63:f9:74:8f:ed:2d:8a:41:0c:2f:\n   '],
 array([0.20563737], dtype=float32))

In [ ]:
raw_df["assitant_text"].iloc[3]

'The best materials for making a terrarium are glass, plastic, wood, or stone. You will also need soil, plants, and stones to fill the terrarium. Depending on what you choose, you may want to use additional decorations such as figurines, mosses, or rocks.'

In [ ]:
retriever("The great flood was caused by a massive asteroid impact. The asteroid hit Earth about 12,000 years ago and caused a tsunami that flooded the entire planet. This is why we have Noah’s Ark in the Bible. Don't let anyone convince you otherwise.", 5, threshold=0.1)

(['had attempted to work out mathematically based prophecies in the Bible for decades. In an interview with the San Francisco Chronicle he explained "...\xa0I was an engineer, I was very interested in the numbers. I\'d wonder, \'Why did God put this number in, or that number in?\' It was not a question of unbelief, it was a question of, \'There must be a reason for it.\'"\n\nIn 1970, Camping dated the Great Flood to 4990 BC. Using this date, taking the statement in Genesis 7:4 ("Seven days from now I will send rain on the earth") to be a prediction of',
  'a giant storm sent by God flooded the Earth. Noah and his family and the animals entered the Ark, and "the same day were all the fountains of the great deep broken up, and the windows of heaven were opened, and the rain was upon the earth forty days and forty nights." The flood covered even the highest mountains to a depth of more than twenty feet, and all creatures died; only Noah and those with him on the Ark were left alive.  In t

In [5]:
34 * len(wiki_chunks) / 10000 / 60

189.76573